In [92]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from scipy import stats
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import r2_score

## Combine all Data sets

In [28]:
df_hc = pd.read_csv('..\..\Go_Daddy_Data\Data\Data\RDC_Inventory_Core_Metrics_County_History.csv') #Done
df_bls = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\bls-emp_by_cfips.xlsx') #Done
df_ctr = pd.read_csv(r'..\..\Go_Daddy_Data\Data\Data\Corporate_tax_rate.csv') #Done
df_rm_FY17 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2017_50_rev.xlsx') #Done
df_rm_FY18 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2018_50_County_rev.xlsx') #Done
df_rm_FY19 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2019_50_County_rev.xlsx') #Done
df_rm_FY20 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2020_50_County_rev.xlsx') #Done
df_rm_FY21 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2021_50_County.xlsx') #Done
df_rm_FY22 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2022_FMR_50_county_rev.xlsx') #Done
df_rm_FY23 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2023_FMR_50_county.xlsx') #Done
df_me = pd.read_csv(r'..\..\Go_Daddy_Data\Data\Data\Medicare_Monthly_Enrollment_September_2022.csv') #Done


df_cs = pd.read_csv('..\Data\data_zip\census_starter.csv') #Done
df_test = pd.read_csv(r'..\Data\data_zip\test.csv')
df_train = pd.read_csv(r'..\Data\data_zip\train.csv')

In [29]:
df_train['first_day_of_month'] = pd.to_datetime(df_train['first_day_of_month'])
df_train['year'] = df_train['first_day_of_month'].dt.year

In [30]:
df_bls2 = df_bls[df_bls['period'] != 'M13']
df_bls2_2 = pd.pivot_table(df_bls2, values='value', index=['cfips', 'first_day_of_month'], columns=['measure_type'], aggfunc=np.sum).reset_index()
df_bls2_2['first_day_of_month'] = pd.to_datetime(df_bls2_2['first_day_of_month'])
df_bls2_2['first_day_of_month_lag_12'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=12)
#df_bls2_2 = df_bls2_2.drop(columns=['Labor force'])
df_bls2_2
df_train_comb = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_12']
                               , suffixes=['_orig', '_bls'])
df_train_comb

,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,year,first_day_of_month_bls,Employment,Labor force,Unemployment,Unemployment rate,first_day_of_month_lag_12
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,26496.0,1012.0,3.8,2019-08-01
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,2018-09-01,25443.0,26354.0,911.0,3.5,2019-09-01
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,2018-10-01,25614.0,26526.0,912.0,3.4,2019-10-01
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,2018-11-01,25639.0,26485.0,846.0,3.2,2019-11-01
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,2018-12-01,25664.0,26543.0,879.0,3.3,2019-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,NaT,NaN,NaN,NaN,NaN,NaT
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,NaT,NaN,NaN,NaN,NaN,NaT
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,NaT,NaN,NaN,NaN,NaN,NaT
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,NaT,NaN,NaN,NaN,NaN,NaT


In [31]:
df_cs2 = pd.melt(df_cs, id_vars=['cfips'])
df_cs2['Year'] = df_cs2['variable'].str[-4:]
df_cs2['variable'] = df_cs2['variable'].str[:-5]
df_cs2.sort_values(['cfips']).reset_index(drop=True)
df_cs2 = pd.pivot_table(df_cs2, values='value', index=['cfips', 'Year'], columns=['variable'], aggfunc=np.sum).reset_index()
df_cs2['year_lag'] = (pd.to_datetime(df_cs2['Year']) + pd.DateOffset(years=2)).dt.year
#df_cs2 = df_cs2.drop(columns=['median_hh_inc'])
df_train_comb = df_train_comb.merge(df_cs2, how='left', left_on=['cfips', 'year'], right_on=['cfips','year_lag'], 
                              suffixes=['_orig', '_cps'])
df_train_comb

,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,year,first_day_of_month_bls,Employment,...,Unemployment,Unemployment rate,first_day_of_month_lag_12,Year,median_hh_inc,pct_bb,pct_college,pct_foreign_born,pct_it_workers,year_lag
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,...,1012.0,3.8,2019-08-01,2017,55317.0,76.6,14.5,2.1,1.3,2019
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,2018-09-01,25443.0,...,911.0,3.5,2019-09-01,2017,55317.0,76.6,14.5,2.1,1.3,2019
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,2018-10-01,25614.0,...,912.0,3.4,2019-10-01,2017,55317.0,76.6,14.5,2.1,1.3,2019
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,2018-11-01,25639.0,...,846.0,3.2,2019-11-01,2017,55317.0,76.6,14.5,2.1,1.3,2019
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,2018-12-01,25664.0,...,879.0,3.3,2019-12-01,2017,55317.0,76.6,14.5,2.1,1.3,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,NaT,NaN,...,NaN,NaN,NaT,2020,53333.0,79.7,12.7,2.3,0.0,2022
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,NaT,NaN,...,NaN,NaN,NaT,2020,53333.0,79.7,12.7,2.3,0.0,2022
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,NaT,NaN,...,NaN,NaN,NaT,2020,53333.0,79.7,12.7,2.3,0.0,2022
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,NaT,NaN,...,NaN,NaN,NaT,2020,53333.0,79.7,12.7,2.3,0.0,2022


In [32]:
df_hc['date'] = pd.to_datetime(df_hc['month_date_yyyymm'].apply(str).str[0:4]+'-'\
                               +df_hc['month_date_yyyymm'].apply(str).str[4:6]+'-01')
keep = ['date', 'county_fips', 'active_listing_count', 'active_listing_count_yy', 'median_days_on_market', 
        'new_listing_count', 'price_reduced_count']
df_hc_vars = df_hc[keep]
df_hc_vars['date_lag_12'] = df_hc_vars['date'] + pd.DateOffset(months=12)
df_train_comb = df_train_comb.merge(df_hc_vars, how='left', left_on=['cfips', 'first_day_of_month_orig'], \
                                    right_on=['county_fips','date_lag_12'], suffixes=['_orig', '_hc'])
df_train_comb

C:\Users\jacks\AppData\Local\Temp\ipykernel_35680\571208765.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hc_vars['date_lag_12'] = df_hc_vars['date'] + pd.DateOffset(months=12)


,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,year,first_day_of_month_bls,Employment,...,pct_it_workers,year_lag,date,county_fips,active_listing_count,active_listing_count_yy,median_days_on_market,new_listing_count,price_reduced_count,date_lag_12
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,...,1.3,2019,2018-08-01,1001.0,226.0,0.0416,76.0,66.0,56.0,2019-08-01
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,2018-09-01,25443.0,...,1.3,2019,2018-09-01,1001.0,221.0,-0.0391,76.0,88.0,56.0,2019-09-01
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,2018-10-01,25614.0,...,1.3,2019,2018-10-01,1001.0,202.0,-0.1534,76.0,60.0,54.0,2019-10-01
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,2018-11-01,25639.0,...,1.3,2019,2018-11-01,1001.0,201.0,-0.1206,74.0,62.0,32.0,2019-11-01
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,2018-12-01,25664.0,...,1.3,2019,2018-12-01,1001.0,200.0,-0.0566,76.0,80.0,24.0,2019-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,NaT,NaN,...,0.0,2022,2021-06-01,56045.0,45.0,-0.2458,63.0,12.0,0.0,2022-06-01
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,NaT,NaN,...,0.0,2022,2021-07-01,56045.0,50.0,-0.2063,58.0,24.0,12.0,2022-07-01
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,NaT,NaN,...,0.0,2022,2021-08-01,56045.0,62.0,-0.0238,62.0,12.0,8.0,2022-08-01
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,NaT,NaN,...,0.0,2022,2021-09-01,56045.0,55.0,-0.0265,75.0,10.0,10.0,2022-09-01


In [33]:
df_me_yearly = df_me[(df_me['MONTH'] == 'Year') & (df_me['BENE_GEO_LVL'] == 'County')]
df_me_yearly.BENE_FIPS_CD = df_me_yearly.BENE_FIPS_CD.astype(int)

df_me_monthly = df_me[(df_me['MONTH'] != 'Year') & (df_me['BENE_GEO_LVL'] == 'County')]
df_me_monthly.BENE_FIPS_CD = df_me_monthly.BENE_FIPS_CD.astype(int)

df_me_monthly['date'] = pd.to_datetime(df_me_monthly['YEAR'].astype(str)  + df_me_monthly['MONTH'], format='%Y%B')

C:\Users\jacks\AppData\Local\Temp\ipykernel_35680\2785904398.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me_yearly.BENE_FIPS_CD = df_me_yearly.BENE_FIPS_CD.astype(int)
C:\Users\jacks\AppData\Local\Temp\ipykernel_35680\2785904398.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me_monthly.BENE_FIPS_CD = df_me_monthly.BENE_FIPS_CD.astype(int)
C:\Users\jacks\AppData\Local\Temp\ipykernel_35680\2785904398.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [34]:
date_df = pd.DataFrame(pd.date_range(start='2017-01-01', end='2021-08-01'), columns=['date'])

date_df['year'] = date_df.date.dt.year
date_df['day'] = date_df.date.dt.day
date_df = date_df[date_df.day == 1]

df_me_yearly_month = df_me_yearly.merge(date_df, how="left", left_on='YEAR', right_on='year')
df_me_yearly_month = df_me_yearly_month.drop(['year', 'day'], axis=1)
df_me_yearly_month

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES,date
0,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-01-01
1,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-02-01
2,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-03-01
3,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-04-01
4,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183423,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-04-01
183424,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-05-01
183425,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-06-01
183426,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-07-01


In [35]:
df_me_final = pd.concat([df_me_monthly, df_me_yearly_month])
df_me_final = df_me_final.replace({'*': np.nan})
df_me_final['date_lag_12'] = df_me_final['date'] + pd.DateOffset(months=12)
keep = ['BENE_FIPS_CD','date_lag_12', 'TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES', 'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES']
df_me_final_keep = df_me_final[keep]
me_values = ['TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES',
       'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES']
for i in me_values:
    df_me_final_keep[i] = df_me_final_keep[i].astype(float)

#df_me_final_keep_dup = df_me_final_keep[df_me_final_keep[['BENE_FIPS_CD','date_lag_12']].duplicated()]
print(len(df_train))
df_train_comb = df_train_comb.merge(df_me_final_keep, how='left', left_on=['cfips', 'first_day_of_month_orig'], right_on=['BENE_FIPS_CD','date_lag_12'], 
                              suffixes=['_orig', '_me'])
df_train_comb

C:\Users\jacks\AppData\Local\Temp\ipykernel_35680\2600750921.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me_final_keep[i] = df_me_final_keep[i].astype(float)


122265


,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,year,first_day_of_month_bls,Employment,...,new_listing_count,price_reduced_count,date_lag_12_orig,BENE_FIPS_CD,date_lag_12_me,TOT_BENES,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,...,66.0,56.0,2019-08-01,1001.0,2019-08-01,10645.0,8241.0,2403.0,9892.0,7088.0
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,2018-09-01,25443.0,...,88.0,56.0,2019-09-01,1001.0,2019-09-01,10645.0,8241.0,2403.0,9892.0,7088.0
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,2018-10-01,25614.0,...,60.0,54.0,2019-10-01,1001.0,2019-10-01,10645.0,8241.0,2403.0,9892.0,7088.0
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,2018-11-01,25639.0,...,62.0,32.0,2019-11-01,1001.0,2019-11-01,10645.0,8241.0,2403.0,9892.0,7088.0
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,2018-12-01,25664.0,...,80.0,24.0,2019-12-01,1001.0,2019-12-01,10645.0,8241.0,2403.0,9892.0,7088.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,NaT,NaN,...,12.0,0.0,2022-06-01,56045.0,2022-06-01,1713.0,1555.0,158.0,1588.0,1133.0
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,NaT,NaN,...,24.0,12.0,2022-07-01,56045.0,2022-07-01,1713.0,1555.0,158.0,1588.0,1133.0
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,NaT,NaN,...,12.0,8.0,2022-08-01,56045.0,2022-08-01,1713.0,1555.0,158.0,1588.0,1133.0
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,NaT,NaN,...,10.0,10.0,2022-09-01,56045.0,2022-09-01,1716.0,1555.0,161.0,1591.0,1135.0


In [36]:
df_rm_FY17['fips'] = df_rm_FY17['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY17['year'] = 2017
df_rm_FY17 = df_rm_FY17.rename(columns={'Rent50_0': 'rent50_0', 'Rent50_1': 'rent50_1', 'Rent50_2': 'rent50_2', 'Rent50_3': 'rent50_3', 'Rent50_4': 'rent50_4'})
df_rm_FY17_keep = df_rm_FY17[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY18['fips'] = df_rm_FY18['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY18['year'] = 2018
df_rm_FY18_keep = df_rm_FY18[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY19['fips'] = df_rm_FY19['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY19['year'] = 2019
df_rm_FY19_keep = df_rm_FY19[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY20['fips'] = df_rm_FY20['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY20['year'] = 2020
df_rm_FY20_keep = df_rm_FY20[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY21['fips'] = df_rm_FY21['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY21['year'] = 2021
df_rm_FY21_keep = df_rm_FY21[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY22['fips'] = df_rm_FY22['state_code'].astype(str) + df_rm_FY22['county_code'].astype(str).str.zfill(3)
df_rm_FY22['fips'] = df_rm_FY22['fips'].astype(int)
df_rm_FY22['year'] = 2022
df_rm_FY22 = df_rm_FY22.rename(columns={'rent_50_0': 'rent50_0', 'rent_50_1': 'rent50_1', 'rent_50_2': 'rent50_2', 'rent_50_3': 'rent50_3', 'rent_50_4': 'rent50_4'})
df_rm_FY22_keep = df_rm_FY22[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY23['fips'] = df_rm_FY23['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY23['year'] = 2023
df_rm_FY23 = df_rm_FY23.rename(columns={'rent_50_0': 'rent50_0', 'rent_50_1': 'rent50_1', 'rent_50_2': 'rent50_2', 'rent_50_3': 'rent50_3', 'rent_50_4': 'rent50_4'})
df_rm_FY23_keep = df_rm_FY23[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm = pd.concat([df_rm_FY17_keep, df_rm_FY18_keep, df_rm_FY19_keep, df_rm_FY20_keep, df_rm_FY21_keep, df_rm_FY22_keep, df_rm_FY23_keep])

print(len(df_train))

print(len(df_rm))
df_rm_avg = df_rm.groupby(['fips', 'year']).mean().reset_index()

df_train_comb = df_train_comb.merge(df_rm_avg, how='left', left_on=['cfips', 'year'], right_on=['fips','year'], 
                              suffixes=['_orig', 'rm'])

df_train_comb

122265
33369


,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,year,first_day_of_month_bls,Employment,...,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES,fips,rent50_0,rent50_1,rent50_2,rent50_3,rent50_4
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,...,8241.0,2403.0,9892.0,7088.0,1001,620.0,743.0,880.0,1112.0,1532.0
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,2018-09-01,25443.0,...,8241.0,2403.0,9892.0,7088.0,1001,620.0,743.0,880.0,1112.0,1532.0
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,2018-10-01,25614.0,...,8241.0,2403.0,9892.0,7088.0,1001,620.0,743.0,880.0,1112.0,1532.0
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,2018-11-01,25639.0,...,8241.0,2403.0,9892.0,7088.0,1001,620.0,743.0,880.0,1112.0,1532.0
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,2018-12-01,25664.0,...,8241.0,2403.0,9892.0,7088.0,1001,620.0,743.0,880.0,1112.0,1532.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,NaT,NaN,...,1555.0,158.0,1588.0,1133.0,56045,752.0,757.0,996.0,1231.0,1705.0
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,NaT,NaN,...,1555.0,158.0,1588.0,1133.0,56045,752.0,757.0,996.0,1231.0,1705.0
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,NaT,NaN,...,1555.0,158.0,1588.0,1133.0,56045,752.0,757.0,996.0,1231.0,1705.0
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,NaT,NaN,...,1555.0,161.0,1591.0,1135.0,56045,752.0,757.0,996.0,1231.0,1705.0


In [37]:
df_ctr2 = pd.melt(df_ctr, id_vars=['State'])
df_ctr2['value'] = df_ctr2['value'].str.rstrip('%').astype('float') / 100.0
df_ctr2['year_lag_1'] = (pd.to_datetime(df_ctr2['variable']) + pd.DateOffset(years=1)).dt.year
df_ctr2['State'] = df_ctr2['State'].str[:-1]
df_train_comb = df_train_comb.merge(df_ctr2, how='left', left_on=['state', 'year'], right_on=['State','year_lag_1'], 
                              suffixes=['_orig', 'ctr'])
df_train_comb

,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,year,first_day_of_month_bls,Employment,...,fips,rent50_0,rent50_1,rent50_2,rent50_3,rent50_4,State,variable,value,year_lag_1
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2018-08-01,25484.0,...,1001,620.0,743.0,880.0,1112.0,1532.0,Alabama,2018,0.0514,2019.0
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,2018-09-01,25443.0,...,1001,620.0,743.0,880.0,1112.0,1532.0,Alabama,2018,0.0514,2019.0
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,2018-10-01,25614.0,...,1001,620.0,743.0,880.0,1112.0,1532.0,Alabama,2018,0.0514,2019.0
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,2018-11-01,25639.0,...,1001,620.0,743.0,880.0,1112.0,1532.0,Alabama,2018,0.0514,2019.0
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,2018-12-01,25664.0,...,1001,620.0,743.0,880.0,1112.0,1532.0,Alabama,2018,0.0514,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,NaT,NaN,...,56045,752.0,757.0,996.0,1231.0,1705.0,Wyoming,2021,0.0000,2022.0
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,NaT,NaN,...,56045,752.0,757.0,996.0,1231.0,1705.0,Wyoming,2021,0.0000,2022.0
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,NaT,NaN,...,56045,752.0,757.0,996.0,1231.0,1705.0,Wyoming,2021,0.0000,2022.0
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,NaT,NaN,...,56045,752.0,757.0,996.0,1231.0,1705.0,Wyoming,2021,0.0000,2022.0


In [39]:
df_train_comb.columns

Index(['row_id', 'cfips', 'county', 'state', 'first_day_of_month_orig',
       'microbusiness_density', 'active', 'year', 'first_day_of_month_bls',
       'Employment', 'Labor force', 'Unemployment', 'Unemployment rate',
       'first_day_of_month_lag_12', 'Year', 'median_hh_inc', 'pct_bb',
       'pct_college', 'pct_foreign_born', 'pct_it_workers', 'year_lag', 'date',
       'county_fips', 'active_listing_count', 'active_listing_count_yy',
       'median_days_on_market', 'new_listing_count', 'price_reduced_count',
       'date_lag_12_orig', 'BENE_FIPS_CD', 'date_lag_12_me', 'TOT_BENES',
       'AGED_TOT_BENES', 'DSBLD_TOT_BENES', 'A_B_TOT_BENES',
       'PRSCRPTN_DRUG_TOT_BENES', 'fips', 'rent50_0', 'rent50_1', 'rent50_2',
       'rent50_3', 'rent50_4', 'State', 'variable', 'value', 'year_lag_1'],
      dtype='object')

In [87]:
df_train_comb_reg = df_train_comb[['row_id', 'cfips', 'county', 'state', 'first_day_of_month_orig',
       'microbusiness_density', 'active',
       'Employment', 'Labor force', 'Unemployment', 'Unemployment rate',
       'median_hh_inc', 'pct_bb',
       'pct_college', 'pct_foreign_born', 'pct_it_workers', 'active_listing_count', 'active_listing_count_yy',
       'median_days_on_market', 'new_listing_count', 'price_reduced_count',
       'TOT_BENES',
       'AGED_TOT_BENES', 'DSBLD_TOT_BENES', 'A_B_TOT_BENES',
       'PRSCRPTN_DRUG_TOT_BENES', 'rent50_0', 'rent50_1', 'rent50_2',
       'rent50_3', 'rent50_4','value']]
x_values = df_train_comb_reg[['Employment', 'Labor force', 'Unemployment', 'Unemployment rate',
       'median_hh_inc', 'pct_bb',
       'pct_college', 'pct_foreign_born', 'pct_it_workers', 'active_listing_count', 'active_listing_count_yy',
       'median_days_on_market', 'new_listing_count', 'price_reduced_count',
       'TOT_BENES',
       'AGED_TOT_BENES', 'DSBLD_TOT_BENES', 'A_B_TOT_BENES',
       'PRSCRPTN_DRUG_TOT_BENES', 'rent50_0', 'rent50_1', 'rent50_2',
       'rent50_3', 'rent50_4','value']]
y_value = df_train_comb_reg['microbusiness_density']
x_values_cols = ['Employment', 'Labor force', 'Unemployment', 'Unemployment rate',
       'median_hh_inc', 'pct_bb',
       'pct_college', 'pct_foreign_born', 'pct_it_workers', 'active_listing_count', 'active_listing_count_yy',
       'median_days_on_market', 'new_listing_count', 'price_reduced_count',
       'TOT_BENES',
       'AGED_TOT_BENES', 'DSBLD_TOT_BENES', 'A_B_TOT_BENES',
       'PRSCRPTN_DRUG_TOT_BENES', 'rent50_0', 'rent50_1', 'rent50_2',
       'rent50_3', 'rent50_4','value']
df_train_comb_reg

,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,Employment,Labor force,Unemployment,...,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES,rent50_0,rent50_1,rent50_2,rent50_3,rent50_4,value
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,25484.0,26496.0,1012.0,...,8241.0,2403.0,9892.0,7088.0,620.0,743.0,880.0,1112.0,1532.0,0.0514
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,25443.0,26354.0,911.0,...,8241.0,2403.0,9892.0,7088.0,620.0,743.0,880.0,1112.0,1532.0,0.0514
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,25614.0,26526.0,912.0,...,8241.0,2403.0,9892.0,7088.0,620.0,743.0,880.0,1112.0,1532.0,0.0514
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,25639.0,26485.0,846.0,...,8241.0,2403.0,9892.0,7088.0,620.0,743.0,880.0,1112.0,1532.0,0.0514
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,25664.0,26543.0,879.0,...,8241.0,2403.0,9892.0,7088.0,620.0,743.0,880.0,1112.0,1532.0,0.0514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,NaN,NaN,NaN,...,1555.0,158.0,1588.0,1133.0,752.0,757.0,996.0,1231.0,1705.0,0.0000
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,NaN,NaN,NaN,...,1555.0,158.0,1588.0,1133.0,752.0,757.0,996.0,1231.0,1705.0,0.0000
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,NaN,NaN,NaN,...,1555.0,158.0,1588.0,1133.0,752.0,757.0,996.0,1231.0,1705.0,0.0000
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,NaN,NaN,NaN,...,1555.0,161.0,1591.0,1135.0,752.0,757.0,996.0,1231.0,1705.0,0.0000


In [129]:
imp = SimpleImputer(strategy='constant', fill_value=0)
x_values = imp.fit_transform(x_values)
x_values = pd.DataFrame(x_values, columns=x_values_cols)
x_values

,Employment,Labor force,Unemployment,Unemployment rate,median_hh_inc,pct_bb,pct_college,pct_foreign_born,pct_it_workers,active_listing_count,...,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES,rent50_0,rent50_1,rent50_2,rent50_3,rent50_4,value
0,25484.000000,26496.000000,1012.000000,3.800000,55317.0,76.6,14.5,2.1,1.3,226.0,...,8241.0,2403.0,9892.0,7088.0,620.0,743.0,880.0,1112.0,1532.0,0.0514
1,25443.000000,26354.000000,911.000000,3.500000,55317.0,76.6,14.5,2.1,1.3,221.0,...,8241.0,2403.0,9892.0,7088.0,620.0,743.0,880.0,1112.0,1532.0,0.0514
2,25614.000000,26526.000000,912.000000,3.400000,55317.0,76.6,14.5,2.1,1.3,202.0,...,8241.0,2403.0,9892.0,7088.0,620.0,743.0,880.0,1112.0,1532.0,0.0514
3,25639.000000,26485.000000,846.000000,3.200000,55317.0,76.6,14.5,2.1,1.3,201.0,...,8241.0,2403.0,9892.0,7088.0,620.0,743.0,880.0,1112.0,1532.0,0.0514
4,25664.000000,26543.000000,879.000000,3.300000,55317.0,76.6,14.5,2.1,1.3,200.0,...,8241.0,2403.0,9892.0,7088.0,620.0,743.0,880.0,1112.0,1532.0,0.0514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,81045.093335,86054.410758,5001.904732,5.249164,53333.0,79.7,12.7,2.3,0.0,45.0,...,1555.0,158.0,1588.0,1133.0,752.0,757.0,996.0,1231.0,1705.0,0.0000
122261,81045.093335,86054.410758,5001.904732,5.249164,53333.0,79.7,12.7,2.3,0.0,50.0,...,1555.0,158.0,1588.0,1133.0,752.0,757.0,996.0,1231.0,1705.0,0.0000
122262,81045.093335,86054.410758,5001.904732,5.249164,53333.0,79.7,12.7,2.3,0.0,62.0,...,1555.0,158.0,1588.0,1133.0,752.0,757.0,996.0,1231.0,1705.0,0.0000
122263,81045.093335,86054.410758,5001.904732,5.249164,53333.0,79.7,12.7,2.3,0.0,55.0,...,1555.0,161.0,1591.0,1135.0,752.0,757.0,996.0,1231.0,1705.0,0.0000


In [130]:
#Copied needs to be reworked

lm = LinearRegression(positive=True)
lm.fit(x_values,y_value)
params = np.append(lm.intercept_,lm.coef_)
predictions = lm.predict(x_values)

newX = np.append(np.ones((len(x_values),1)), x_values, axis=1)
MSE = (sum((y_value-predictions)**2))/(len(newX)-len(newX[0]))

# Note if you don't want to use a DataFrame replace the two lines above with
# newX = np.append(np.ones((len(X),1)), X, axis=1)
# MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
sd_b = np.sqrt(var_b)
ts_b = params/ sd_b

#print(newX[0])
'''
for i in ts_b:
    print(len(newX))
    print(len(newX[0]))
    print(len(newX)-len(newX[0]))
    print(stats.t.cdf(np.abs(i),(len(newX)-len(newX[0]))))
'''    
p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-len(newX[0])))) for i in ts_b]

sd_b = np.round(sd_b,3)
ts_b = np.round(ts_b,3)
p_values = np.round(p_values,3)
params = np.round(params,4)

myDF3 = pd.DataFrame()
myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilities"] = [params,sd_b,ts_b,p_values]
myDF3.index = ['const'] + x_values_cols
print(r2_score(y_value, predictions))
myDF3

0.2816223339611429


,Coefficients,Standard Errors,t values,Probabilities
const,-4.0674,0.135,-30.066,0.000
Employment,0.0000,0.000,0.000,1.000
Labor force,0.0000,0.000,0.000,1.000
Unemployment,0.0000,0.000,0.000,1.000
Unemployment rate,0.0000,0.010,0.000,1.000
median_hh_inc,0.0000,0.000,0.000,1.000
pct_bb,0.0106,0.002,4.975,0.000
pct_college,0.2685,0.003,80.355,0.000
pct_foreign_born,0.0064,0.003,2.266,0.023
pct_it_workers,0.2339,0.014,16.713,0.000


In [131]:
# keep pct_bb, pct_college, pct_foreign_born, pct_it_workers, active_listing_count, rent50_3, rent50_4

In [132]:
x = sm.add_constant(x_values) # adding a constant

model = sm.OLS(y_value, x).fit()

print(model.summary())

                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.292
Model:                               OLS   Adj. R-squared:                  0.292
Method:                    Least Squares   F-statistic:                     2016.
Date:                   Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                           17:23:48   Log-Likelihood:            -3.4894e+05
No. Observations:                 122265   AIC:                         6.979e+05
Df Residuals:                     122239   BIC:                         6.982e+05
Df Model:                             25                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [94]:
#Step-wise ANOVA
#removed median_days_on_market
x = sm.add_constant(x_values.drop(columns=['median_days_on_market'])) # adding a constant

model = sm.OLS(y_value, x).fit()

print(model.summary())

                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.292
Model:                               OLS   Adj. R-squared:                  0.292
Method:                    Least Squares   F-statistic:                     2100.
Date:                   Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                           16:44:58   Log-Likelihood:            -3.4894e+05
No. Observations:                 122265   AIC:                         6.979e+05
Df Residuals:                     122240   BIC:                         6.982e+05
Df Model:                             24                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [95]:
#Step-wise ANOVA
#removed rent50_0
x = sm.add_constant(x_values.drop(columns=['median_days_on_market', 'rent50_0'])) # adding a constant

model = sm.OLS(y_value, x).fit()

print(model.summary())

                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.292
Model:                               OLS   Adj. R-squared:                  0.292
Method:                    Least Squares   F-statistic:                     2191.
Date:                   Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                           16:48:44   Log-Likelihood:            -3.4894e+05
No. Observations:                 122265   AIC:                         6.979e+05
Df Residuals:                     122241   BIC:                         6.982e+05
Df Model:                             23                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [96]:
#Step-wise ANOVA
#removed Unemployment
x = sm.add_constant(x_values.drop(columns=['median_days_on_market', 'rent50_0', 'Unemployment'])) # adding a constant

model = sm.OLS(y_value, x).fit()

print(model.summary())

                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.292
Model:                               OLS   Adj. R-squared:                  0.292
Method:                    Least Squares   F-statistic:                     2291.
Date:                   Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                           16:49:46   Log-Likelihood:            -3.4894e+05
No. Observations:                 122265   AIC:                         6.979e+05
Df Residuals:                     122242   BIC:                         6.982e+05
Df Model:                             22                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [97]:
#Step-wise ANOVA
#removed Unemployment
x = sm.add_constant(x_values.drop(columns=['median_days_on_market', 'rent50_0', 'Unemployment', 'Employment'])) # adding a constant

model = sm.OLS(y_value, x).fit()

print(model.summary())

                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.292
Model:                               OLS   Adj. R-squared:                  0.292
Method:                    Least Squares   F-statistic:                     2400.
Date:                   Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                           16:50:40   Log-Likelihood:            -3.4894e+05
No. Observations:                 122265   AIC:                         6.979e+05
Df Residuals:                     122243   BIC:                         6.981e+05
Df Model:                             21                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [98]:
#Step-wise ANOVA
#removed Unemployment rate
x = sm.add_constant(x_values.drop(columns=['median_days_on_market', 'rent50_0', 'Unemployment', 'Employment', 'Unemployment rate'])) # adding a constant

model = sm.OLS(y_value, x).fit()

print(model.summary())

                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.292
Model:                               OLS   Adj. R-squared:                  0.292
Method:                    Least Squares   F-statistic:                     2520.
Date:                   Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                           16:51:18   Log-Likelihood:            -3.4894e+05
No. Observations:                 122265   AIC:                         6.979e+05
Df Residuals:                     122244   BIC:                         6.981e+05
Df Model:                             20                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [99]:
#Step-wise ANOVA
#removed pct_foreign_born
x = sm.add_constant(x_values.drop(columns=['median_days_on_market', 'rent50_0', 'Unemployment', 'Employment', 'Unemployment rate', 'pct_foreign_born'])) # adding a constant

model = sm.OLS(y_value, x).fit()

print(model.summary())

                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.292
Model:                               OLS   Adj. R-squared:                  0.292
Method:                    Least Squares   F-statistic:                     2652.
Date:                   Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                           16:51:58   Log-Likelihood:            -3.4894e+05
No. Observations:                 122265   AIC:                         6.979e+05
Df Residuals:                     122245   BIC:                         6.981e+05
Df Model:                             19                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [100]:
#Step-wise ANOVA
#removed active_listing_count_yy
x = sm.add_constant(x_values.drop(columns=['median_days_on_market', 'rent50_0', 'Unemployment', 'Employment', 
                                           'Unemployment rate', 'pct_foreign_born', 'active_listing_count_yy'])) # adding a constant

model = sm.OLS(y_value, x).fit()

print(model.summary())

                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.292
Model:                               OLS   Adj. R-squared:                  0.292
Method:                    Least Squares   F-statistic:                     2797.
Date:                   Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                           16:53:04   Log-Likelihood:            -3.4896e+05
No. Observations:                 122265   AIC:                         6.980e+05
Df Residuals:                     122246   BIC:                         6.981e+05
Df Model:                             18                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [101]:
#Step-wise ANOVA
#removed TOT_BENES
x = sm.add_constant(x_values.drop(columns=['median_days_on_market', 'rent50_0', 'Unemployment', 'Employment', 
                                           'Unemployment rate', 'pct_foreign_born', 'active_listing_count_yy', 'TOT_BENES'])) # adding a constant

model = sm.OLS(y_value, x).fit()

print(model.summary())

                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.292
Model:                               OLS   Adj. R-squared:                  0.292
Method:                    Least Squares   F-statistic:                     2961.
Date:                   Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                           16:55:25   Log-Likelihood:            -3.4897e+05
No. Observations:                 122265   AIC:                         6.980e+05
Df Residuals:                     122247   BIC:                         6.981e+05
Df Model:                             17                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [119]:
#Step-wise ANOVA
#removed new_listing_count
x = sm.add_constant(x_values.drop(columns=['median_days_on_market', 'rent50_0', 'Unemployment', 'Employment', 
                                           'Unemployment rate', 'pct_foreign_born', 'active_listing_count_yy', 'TOT_BENES'])) # adding a constant

model = sm.OLS(y_value, x).fit()

print(model.summary())

                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.292
Model:                               OLS   Adj. R-squared:                  0.292
Method:                    Least Squares   F-statistic:                     2961.
Date:                   Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                           16:59:53   Log-Likelihood:            -3.4897e+05
No. Observations:                 122265   AIC:                         6.980e+05
Df Residuals:                     122247   BIC:                         6.981e+05
Df Model:                             17                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

drop:
<br>
'median_days_on_market', 'rent50_0', 'Unemployment', 'Employment', 
'Unemployment rate', 'pct_foreign_born', 'active_listing_count_yy', 'TOT_BENES'
<br>
keep:
<br>
    pct_bb, pct_college, pct_foreign_born, pct_it_workers, active_listing_count, rent50_3, rent50_4
<br>
Final drop:
<br>
    median_days_on_market, rent50_0, Unemployment, Employment, Unemployment rate, active_listing_count_yy, TOT_BENES

In [133]:
df_train_comb_reg.to_csv('training.csv')